In [1]:
import os
import copy
import time
import queue
import random
from IPython.display import clear_output

In [ ]:
from onedrivedownloader import download

link2data = 'https://unioulu-my.sharepoint.com/:u:/g/personal/jukmaatt_univ_yo_oulu_fi/ESs3tMxzs1JOp7-IfspuD24BC53tZacALcyeejw5DF6T_Q?e=JdnssV'

if not os.path.exists('./kuvat'):
    print('Downloading data')
    download(link2data, filename='./kuvat.zip', unzip=True, unzip_path='./')

# <center>521160P Johdatus Tekoälyyn<br><br>Harjoitus #1<br><br>Hakualgoritmit<br>

Tässä harjoituksessa käsitellään syvyyshakua ja leveyshakua joenylitysongelman ratkaisemiseksi ja luodaan heuristisia arviointifunktioita 8-puzzle pelille. **Katso harjoitukseen liittyvät palautuksen deadline moodlesta**. Harjoituksesta on mahdollisuus saada 4 pistettä (2.0p + 2.0p).

Jos kohtaat ongelmia harjoitusta tehdessäsi tai sinulla on kysyttävää harjoituksien käytäntöihin liittyen, ole yhteydessä **Moodlesta** löytyvän **ohjelmointiharjoitus 1** foorumin kautta.

**Täytä aluksi ryhmäsi jäsenten tiedot (nimi ja opiskelijanumero)**

# Ryhmän jäsenten tiedot:

* **Jäsen 1 :** `Matti Meikäläinen 1234567 `
* **Jäsen 2 :** `Maija Meikäläinen 2345678 `
* **Jäsen 3 :** `... `

# Tehtävä 1. Susi, lammas, kaali ja paimen joenylitysongelma syvyyshaulla ja leveyshaulla

<br>
<div style="width:image width px; font-size:80%; text-align:center;">
    <center>
    <img src='kuvat/rivercrossingpuzzle.png' width='700' height='auto' alt='alternate text' style='padding-bottom:0.5em;' />
    </center>
    <span>Kuva 1. Susi, lammas, kaali ja paimen joenylitysongelman alkutilanne.</span>
</div>
<br>

Ongelman kuvaus: Paimen kuljettaa soutuveneellä meritietä pitkin suden, lampaan ja kaalin saareen. Soutuvene on pieni, joten siihen mahtuu paimenen lisäksi vain yksi objekti kerrallaan. Paimenen tehtävää rajoittaa seuraavat ehdot: 

* Lammasta ei saa jättää yksin kaalin kanssa samalle puolelle, sillä silloin lammas syö kaalin. 

* Sutta ei saa jättää yksin lampaan kanssa samalle puolelle, sillä silloin susi syö lampaan.

Auta paimenta löytämään oikeat liikkeet päästäkseen saareen suden, lampaan ja kaalin kanssa käyttämällä **leveyshakua** ja **syvyyshakua**. Määritetään aluksi peliin osallistuvat objektit sekä kuvan 1 mukainen pelin alkutilanne, jossa kaikki objektit ovat mantereella ja saaressa ei ole ketään.

In [5]:
objektit = sorted(['S','L','K','P']) # sorted() komento huolehtii, että susi, lammas, kaali ja paimen ovat listassa aakkosjärjestyksessä
saari = sorted([])
manner = copy.deepcopy(objektit)
print('Alkutilanteessa mantereella on {} ja saaressa on {}'.format(manner, saari))

Alkutilanteessa mantereella on ['K', 'L', 'P', 'S'] ja saaressa on []


Toteutetaan tämän jälkeen apufunktioita ongelman rajoitteiden määrittämiseksi.

In [3]:
def paimen_saaressa(saari):
    """
    Tässä funktiossa tarkistetaan onko paimen saaressa vai mantereella. Jos paimen on saaressa, palautetaan totuusarvo True.
    Muussa tapauksessa palautetaan totuusarvo False.
    """
    if 'P' in saari:
        return True
    return False

def objektit_saaressa(objektit, manner):
    """
    Tässä funktiossa selvitetään objektit saaressa, kun tiedossa on mitä objekteja on mantereella.
    Poistetaan kaikista peliin osallistuvista objekteista mantereen objektit, jolloin saadaan saaressa olevat objektit.
    """
    return [objekti for objekti in objektit if objekti not in manner]

def laillinen_tila(objektit, manner):
    """
    Tässä funktiossa tarkistetaan, onko toteutettavasta liikkeestä aiheutunut tila laillinen molemmilla puolin meritietä.
    Jos liike on laiton, palautetaan totuusarvo False. Muussa tapauksessa palautetaan totuusarvo True.
    """
    saari = objektit_saaressa(objektit, manner)
    # Jos paimen on saaressa, (susi ja lammas) tai (lammas ja kaali) ei saa olla mantereella.
    if paimen_saaressa(saari):
        if ('S' in manner and 'L' in manner) or ('L' in manner and 'K' in manner):
            return False
        return True
    # Jos paimen on mantereella, (susi ja lammas) tai (lammas ja kaali) ei saa olla saaressa.
    else:
        if ('S' in saari and 'L' in saari) or ('L' in saari and 'K' in saari):
            return False
        return True

def generoi_uudet_lailliset_mantereet(objektit, manner):
    """
    Tässä funktiossa generoidaan uudet lailliset tilat mantereelle.
    """
    manner = copy.deepcopy(manner)
    uudet_mantereet = [] # alustetaan uudet_mantereet lista
    saari = objektit_saaressa(objektit, manner)
    if paimen_saaressa(saari):
        saari.remove('P') # vähintään paimen liikkuu pois saaresta mantereelle
        manner.append('P')
        if laillinen_tila(objektit, manner):
            uudet_mantereet.append(sorted(manner))
        for objekti in saari:
            manner_tmp = manner + [objekti]
            if laillinen_tila(objektit, manner_tmp):
               uudet_mantereet.append(sorted(manner_tmp))
    else:
        manner.remove('P') # vähintään paimen liikkuu pois mantereelta saareen
        saari.append('P')
        if laillinen_tila(objektit, manner):
            uudet_mantereet.append(sorted(manner))
        manner_tmp = copy.deepcopy(manner)
        for objekti in manner:
            manner_tmp.remove(objekti)
            if laillinen_tila(objektit, manner_tmp):
               uudet_mantereet.append(sorted(manner_tmp))
            manner_tmp.append(objekti)
    return uudet_mantereet

def tulosta_liikkeet(objektit, polku):
    """
    Tässä funktiossa tulostetaan tehdyt liikkeet, kun valmis polku on löydetty.
    """
    print('     Manner' + ' '*23 + 'Vene' +' '*28 + 'Saari')
    for i, manner in enumerate(polku):
        saari = objektit_saaressa(objektit, manner)
        if paimen_saaressa(saari):
            print('     {}'.format(manner) + ' '*(61-len(str(manner))) + '{}'.format(saari))
            if i<len(polku)-1:
                vene = [x for x in saari if x not in objektit_saaressa(objektit, polku[i+1])]
                print('{}.   '.format(i+1) + ' '*27 + '<-{}'.format(vene))
        else:
            print('     {}'.format(manner) + ' '*(61-len(str(manner))) + '{}'.format(saari))
            if i<len(polku)-1:
                vene = [x for x in manner if x not in polku[i+1]]
                print('{}.   '.format(i+1) + ' '*29 + '{}->'.format(vene))
    
    

## Sokeat hakustrategiat (engl. uninformed search)

### Syvyyshaku

Toteuta **syvyyshaku** käyttämällä alla olevaa pseudokoodia. Osa pseodokoodin vaiheista on sinulle jo annettuna ja tehtäväsi on toteuttaa puuttuvat vaiheet.
1. Alusta kaksi tyhjää listaa: avoin lista ja suljettu lista. Avoin lista vastaa tutkimattomat tilat listaa ja suljettu lista käsitellyt tilat listaa
2. Määritä alkutila, joka koostuu kahdesta muusta listasta *manner* ja *polku*
3. Lisää alkutila avoimeen listaan
4. Käydään läpi avointa listaa niin kauan, kunnes se on tyhjä
    1. Valitse avoimen listan viimeinen alkio
    2. Jos manner ei ole jo ennestään suljetussa listassa
        1. Lisää manner suljettuun listaan
        2. Tarkasta onko manner sama kuin tavoitetila
            1. Jos näin on, palauta polku
        3. Generoi uudet mantereet käyttämällä apufunktiota *generoi_uudet_lailliset_mantereet(objektit, manner)*
        4. Käy läpi silmukassa uudet mantereet yksi kerrallaan
            1. Selvitä uusi polku lisäämällä nykyiseen polkuun uusi manner
            2. Lisää avoimen listaan uusi tila, joka koostuu läpikäytävästä uudesta mantereesta ja uudesta polusta
5. Tulosta virheilmoitus, kun avoin lista on tyhjä ja ratkaisua ei löytynyt
        
        

In [ ]:
def syvyyshaku(objektit, manner):
    """
    Tässä funktiossa etsitään ongelman ratkaiseva polku käyttämällä syvyyshakua (engl. depth-first-search).
    """
    tavoitetila = []
    polku = [manner]
    #-------- TÄHÄN SINUN KOODI --------
    # 1. Alusta kaksi tyhjää listaa: avoin_lista ja suljettu_lista
    
    
    # 2. Määritetään muuttuja alkutila, joka on lista, joka koostuu kahdesta muusta listasta manner ja polku
    alkutila = [manner,polku]
    # 3. Lisää alkutila avoimeen listaan
    
    # 4. Käy läpi avointa listaa niin kauan, kunnes se on tyhjä (Vihje: while loop)
    
        # 4.A Valitaan avoimen listan viimeinen alkio
        manner, polku = avoin_lista.pop()
        # 4.B Jos muuttuja manner ei ole jo ennestään suljetussa listassa
        
            # 4.B.a Lisää muuttuja manner suljettuun listaan
            
            # 4.B.b Tarkasta onko manner sama kuin tavoitetila
            
                # 4.B.b.i Jos näin on, palauta muuttuja polku
                
            # 4.B.c Generoidaan muuttuja uudet_mantereet käyttämällä apufunktiota generoi_uudet_lailliset_mantereet(objektit, manner)
            uudet_mantereet = generoi_uudet_lailliset_mantereet(objektit, manner)      
            # 4.B.d Käydään läpi silmukassa muuttujan uudet_mantereet alkiot yksi kerrallaan käyttämällä alkiosta nimeä uusi_manner
            for uusi_manner in uudet_mantereet:
                # 4.B.d.i Selvitetään muuttuja uusi_polku lisäämällä listaan polku muuttuja uusi_manner
                uusi_polku = polku + [uusi_manner]
                # 4.B.d.ii Lisätään listaan avoin_lista uusi tila, joka koostuu läpikäytävästä uudesta mantereesta sekä listasta uusi_polku
                avoin_lista.append([uusi_manner, uusi_polku])
    # 5. Tulostetaan virheilmoitus, kun avoin lista on tyhjä ja ratkaisua ei löytynyt
    print('Ratkaisua ei löytynyt')
    #-----------------------------------
    
syvyyshaku_polku = syvyyshaku(objektit, manner)
tulosta_liikkeet(objektit, syvyyshaku_polku)

### Leveyshaku

Toteuta lisäksi **leveyshaku**, jonka ainut ero syvyyshakuun on, että pseudokoodin vaiheessa 4A. valitaan avoimen listan ensimmäinen alkio, kun syvyyshaussa valitaan avoimen listan viimeinen alkio. Osa pseudokoodin vaiheista on sinulle jo annettuna ja tehtäväsi on toteuttaa puuttuvat vaiheet.

**Montako liikettä tavoitetilan löytämiseksi menee leveyshaulla ja syvyyshaulla? Miten ratkaisut eroavat leveyshaun ja syvyyshaun välillä?**

`Tähän sinun vastaus`

In [ ]:
def leveyshaku(objektit, manner):
    """
    Tässä funktiossa etsitään ongelman ratkaiseva polku käyttämällä leveyshaku (engl. breadth-first-search).
    Ratkaisu on muuten samanlainen kuin syvyyshaku, mutta pseudokoodin vaiheessa 4.A valitaan nyt listan avoin_lista ensimmäinen alkio.
    """
    tavoitetila = []
    polku = [manner]    
    #-------- TÄHÄN SINUN KOODI --------   
    # 1. Alusta kaksi tyhjää listaa: avoin_lista ja suljettu_lista
    
    
    # 2. Määritetään muuttuja alkutila, joka on lista, joka koostuu kahdesta muusta listasta manner ja polku
    alkutila = [manner,polku]
    # 3. Lisää alkutila avoimeen listaan
    
    # 4. Käy läpi avointa listaa niin kauan, kunnes se on tyhjä (Vihje: while loop)

        # 4.A Valitse avoimen listan ensimmäinen alkio (Vihje: .pop(0))
    
        # 4.B Jos muuttuja manner ei ole jo ennestään suljetussa listassa
        
            # 4.B.a Lisää muuttuja manner suljettuun listaan
            
            # 4.B.b Tarkasta onko manner sama kuin tavoitetila
            
                # 4.B.b.i Jos näin on, palauta muuttuja polku
                
            # 4.B.c Generoidaan muuttuja uudet_mantereet käyttämällä apufunktiota generoi_uudet_lailliset_mantereet(objektit, manner)
            uudet_mantereet = generoi_uudet_lailliset_mantereet(objektit, manner)            
            # 4.B.d Käydään läpi silmukassa muuttujan uudet_mantereet alkiot yksi kerrallaan käyttämällä alkiosta nimeä uusi_manner
            for uusi_manner in uudet_mantereet:
                # 4.B.d.i Selvitetään muuttuja uusi_polku lisäämällä listaan polku muuttuja uusi_manner
                uusi_polku = polku + [uusi_manner]
                # 4.B.d.ii Lisätään listaan avoin_lista uusi tila, joka koostuu läpikäytävästä uudesta mantereesta sekä listasta uusi_polku
                avoin_lista.append([uusi_manner, uusi_polku])
    # 5. Tulostetaan virheilmoitus, kun avoin lista on tyhjä ja ratkaisua ei löytynyt
    print('Ratkaisua ei löytynyt')
    #-----------------------------------
    
leveyshaku_polku = leveyshaku(objektit, manner)
tulosta_liikkeet(objektit, leveyshaku_polku)

# Tehtävä 2. 8-puzzle peli heuristisilla arviointifunktioilla

<br>
<div style="width:image width px; font-size:80%; text-align:center;">
    <center>
    <img src='kuvat/eightpuzzle1.png' width='400' height='auto' alt='alternate text' style='padding-bottom:0.5em;' />
    </center>
    <span>Kuva 2. Eräs alkutila ja tavoitetila 8-puzzle-pelille.</span>
</div>
<br>

Tässä tehtävässä luodaan heuristisia arviointifunktioita $h(n)$ 8-puzzle pelille. 8-puzzle koostuu kahdeksasta
liukuvasta laatasta, jotka ovat numeroitu ykkösestä kahdeksaan. Numeroidut laatat ovat sijoitettu 3x3
ruudukolle. Yksi ruudukon paikoista jää tyhjäksi, johon voi siirtää minkä tahansa horisontaalisesti tai
vertikaalisesti tyhjän kohdan vieressä olevan laatan. Kun jokin laatoista siirretään tyhjään ruutuun, jää laatan
aiempi paikka tyhjäksi. 8-puzzle pelin tarkoitus on saavuttaa tavoitetila mahdollisimman vähin siirroin
tunnetusta alkutilasta. Kuvassa 2 on 8-puzzle pelin eräs alkutila ja tavoitetila. 

8-puzzle pelin hakuavaruus on niin iso, että sokeilla hakustrategioilla ratkaisun löytäminen kestäisi kauan. Tästä syystä pelin ratkaisun löytämiseksi on käytetty tietoista hakustrategiaa A\*-hakua.  

Toteutetaan seuraavaksi pelin toiminnan kannalta välttämättömiä apufunktioita.

In [6]:
def tulosta_tila(tila):
    """
    Tämä funktio luo tulostettavan pelilaudan.
    """
    tuloste = ''
    tuloste += '+---+---+---+\n'
    for i in range(3):
        for j in range(3):
            laatta = tila[i*3+j]
            tuloste += '| {} '.format(' ' if laatta == 0 else laatta)
        tuloste += '|\n'
        tuloste += '+---+---+---+\n'
    return tuloste
    
def ratkaise_lailliset_liikkeet(sijainti):
    """
    Tämä funktio ratkaisee lailliset liikkeet tyhjälle laatalle.
    """
    liikkeet = [1, -1, 3, -3]
    lailliset_liikkeet = []
    for liike in liikkeet:
        if 0 <= sijainti + liike < 9:
            if liike == 1 and (sijainti==2 or sijainti==5 or sijainti==8):
                continue
            if liike == -1 and (sijainti==0 or sijainti==3 or sijainti==6):
                continue
            lailliset_liikkeet.append(liike)
    return lailliset_liikkeet

def generoi_uudet_lailliset_tilat(tila):
    """
    Tämä funktio generoi seuraavat lailliset tilat nykyisestä tilasta.
    """
    tyhjä_laatta = tila.index(0)   # tyhjä ruutu
    lailliset_liikkeet = ratkaise_lailliset_liikkeet(tyhjä_laatta)
    uudet_tilat = []
    for liike in lailliset_liikkeet:
        uusi_tila = copy.deepcopy(tila)
        (uusi_tila[tyhjä_laatta + liike], uusi_tila[tyhjä_laatta]) = (uusi_tila[tyhjä_laatta], uusi_tila[tyhjä_laatta + liike])
        uudet_tilat.append(uusi_tila)
    return uudet_tilat

def laske_heuristinen_arvo(tila, tavoitetila, heuristinen_funktio):
    """
    Tämä funktio valitsee arviointifunktioksi joko hamming-etäisyyden 'hamming', city-block-etäisyyden 'city-block' tai euklidisen etäisyyden 'euklidinen'.
    """
    if heuristinen_funktio == 'hamming':
        return laske_hamming_etäisyys(tila, tavoitetila)
    elif heuristinen_funktio == 'city-block':
        return laske_city_block_etäisyys(tila, tavoitetila)
    elif heuristinen_funktio == 'euklidinen':
        return laske_euklidinen_etäisyys(tila, tavoitetila)
    else:
        print('Vaihtoehdot ovat \'hamming\' ,\'city-block\' tai \'euklidinen\'')

def järjestä_avoin_lista(avoin_lista, f_arvot):
    """
    Tässä funktiossa järjestetään avoin lista f-arvojen perusteella
    """
    f_arvot_tmp = []
    for tila, polku in avoin_lista:
        f_arvot_tmp.append(f_arvot[string(tila)])
    return [x for y,x in sorted(zip(f_arvot_tmp, avoin_lista))]

def string(lista):
    """
    Tämä funktio muuttaa listan stringiksi.
    """
    return ''.join(list(map(str, lista)))
                
def a_tähti(alkutila, tavoitetila, heuristinen_funktio):
    """
    Tämä funktio ratkaisee pelin käyttämällä A-tähti-hakualgoritmia.
    """
    avoin_lista = []
    suljettu_lista = []
    polku = []
    g_arvot = {}
    f_arvot = {}
    g_arvot[string(alkutila)] = 0
    f_arvot[string(alkutila)] = g_arvot[string(alkutila)] + laske_heuristinen_arvo(alkutila, tavoitetila, heuristinen_funktio)
    tila = [alkutila, polku]
    avoin_lista.append(tila)
    while len(avoin_lista) != 0:
        avoin_lista = järjestä_avoin_lista(avoin_lista, f_arvot)
        tila, polku = avoin_lista.pop(0)
        if tila == tavoitetila:
            uusi_polku = polku + [tila]
            return uusi_polku, suljettu_lista
        suljettu_lista.append(tila)
        uudet_tilat = generoi_uudet_lailliset_tilat(tila)
        for uusi_tila in uudet_tilat:
            alustava_g_arvo = g_arvot[string(tila)] + 1
            if uusi_tila in suljettu_lista and alustava_g_arvo >= g_arvot[string(uusi_tila)]:
                continue
            if uusi_tila not in suljettu_lista or alustava_g_arvo < g_arvot[string(uusi_tila)]:
                uusi_polku = polku + [tila]
                g_arvot[string(uusi_tila)] = alustava_g_arvo
                f_arvot[string(uusi_tila)] = g_arvot[string(uusi_tila)] + laske_heuristinen_arvo(uusi_tila, tavoitetila, heuristinen_funktio)
                if uusi_tila not in avoin_lista:
                    avoin_lista.append([uusi_tila, uusi_polku])
    print('Ratkaisua ei löytynyt!')
    
def ratkaise_peli(alkutila, tavoitetila, arviointifunktio):
    """
    Tämä funktio luo ratkaisun käytetyllä arviointifunktiolla, kun alkutila ja tavoitetila ovat tiedossa.
    """
    polku, suljettu_lista = a_tähti(alkutila, tavoitetila, arviointifunktio)
    for tila in polku:
        clear_output(wait=True)
        print('Alkutila:\n' + tulosta_tila(alkutila))
        print('\nRatkaisu alkutilalle A*-hakualgoritmilla {}-etäisyys arviointifunktiota apuna käyttäen:\n\n'.format(arviointifunktio) + tulosta_tila(tila))
        time.sleep(1)
    print('Ratkaisu sisältää {} liikettä, jonka löytämiseksi tutkittiin yhteensä {} tilaa.'.format(len(polku)-1, len(suljettu_lista)))

## Heuristiset arviointifunktiot

A\*-hakualgoritmi vaatii toimiakseen heuristisen arviointifunktion h(n) eli kustannusarvion tutkittavasta tilasta tavoitetilaan. Tyypillinen ratkaisu pelille on noin 20 liikettä pitkä ja sen haarautumiskerroin (engl. branching factor), eli kuinka monta liikettä keskimäärin on mahdollista suorittaa yhden vuoron aikana, on 2,66. Tämä tarkoittaa sitä, että perusteellinen haku 20 liikkeen syvyydestä käy läpi $2,66^{20} = 3,15*10^{8}$ tilaa. Tästä syystä on erityisen tärkeä löytää mahdollisimman hyvä heuristinen funktio 8-puzzle pelille. Hyviä vaihtoehtoja heuristisiksi funktioiksi ovat **hamming-etäisyys**, **city-block-etäisyys** (tunnetaan myös nimellä manhattan-etäisyys) ja **euklidinen-etäisyys**.

* Hamming-etäisyys: Laskee väärillä paikoilla olevien laattojen lukumäärän. Tila, jolla on vähän laattoja väärillä paikoilla on oletettavasti lähempänä tavoitetilaa kuin esimerkiksi tila, jolla on kaikki laatat väärillä paikoilla.

* City-block-etäisyys: Laskee jokaiselle laatalle horisontaalisen ja vertikaalisen absoluuttisen etäisyyden vastaavasta tavoitetilan laatasta ja summaa lopuksi ne yhteen. $h_{2}(n) = \sum\limits_{i=1}^{8} |a_{ix} - b_{ix}| + |a_{iy} - b_{iy}|$, missä $a_{ix}$ on alkutilan i:nnen laatan x-koordinaatti ja $b_{iy}$ on tavoitetilan i:nnen laatan y-koordinaatti.

* Euklidinen-etäisyys: Laskee jokaiselle laatalle neliöityjen horisontaalisten ja vertikaalisten etäisyyksien summan vastaavasta tavoitetilan laatasta, ottaa summasta neliöjuuren ja lopuksi summaa neliöjuuret yhteen. $h_{3}(n) = \sum\limits_{i=1}^{8} \sqrt{(a_{ix} - b_{ix})^2 + (a_{iy} - b_{iy})^2}$.

Hamming-etäisyys kuvan 2 tilanteelle on $h_{1}(n) = 4$, sillä alkutilan laatoista neljä on väärillä paikoilla
tavoitetilaan verrattuna. City-block-etäisyys lasketaan kuvalle 2 seuraavasti: $h_{2}(n) = 1 + 0 + 0 + 3 + 2 + 0 + 2 + 0 = 8$. Euklidinen-etäisyys puolestaan kuvan 2 tilanteelle on: $h_{3}(n) = 1 + 0 + 0 + \sqrt{5} + \sqrt{2} + 0 + 2 + 0 \approx 6,65$. Alkutilan tyhjällä ruudulla ei ole vaikutusta heurististen funktioiden arvoihin.

A\*-hakualgoritmissa polun kustannusarviona $g(n)$ on käytetty pelipuun syvyyttä. Algoritmissa tutkimattomat tilat tallennetaan avoimeen listaan ja käsitellyt tilat suljettuun listaan. Lopuksi algoritmin löydettyä ratkaisu, rakennetaan polku alkutilasta tavoitetilaan. 

### Euklidinen-etäisyys arviointifunktio

Heuristiset arviointifunktiot ottavat argumentteina tutkittavan tilan ja tavoitetilan. Tutkittava tila ja tavoitetila ovat toteutettu 9 alkiota sisältävinä listoina, missä arvo 0 kuvaa tyhjää ruutua. Listan arvot kulkevat pelilaudan vasemmasta yläkulmasta oikeaan alakulmaan. Kuvan 2 alkutila listana on ***alkutila = [7,2,3,1,0,6,5,8,4]*** ja tavoitetila on ***tavoitetila = [1,2,3,4,5,6,7,8,0]***.

Toteutetaan esimerkkinä heuristinen euklidinen-etäisyys arviointifunktio. Tehtäväsi on toteuttaa hamming-etäisyys arviointifunktio ja city-block-etäisyys arviointifunktio ja testata niiden suorituskykyä kuvan 2 alkutilalle ja tavoitetilalle.

In [7]:
def laske_euklidinen_etäisyys(tila, tavoitetila):
    """
    Tämä funktio laskee euklidisen etäisyyden tilalle tavoitetilasta.
    """
    # Alustetaan muuttuja euklidinen_arvo arvolla 0
    euklidinen_arvo = 0
    # Käydään silmukassa läpi laatan arvot 1-8
    for laatan_arvo in range(1, 9):
        # Selvitetään tilalle ja tavoitetilalle, missä kohtaa matriisia laatan arvo on (esim. tila_x viittaa tilan laatan arvon vaakasuuntaiseen koordinaattiin)
        (tila_x, tila_y) = (tila.index(laatan_arvo) // 3, tila.index(laatan_arvo) % 3)
        (tavoitetila_x, tavoitetila_y) = (tavoitetila.index(laatan_arvo) // 3, tavoitetila.index(laatan_arvo) % 3)
        # Lasketaan jokaiselle laatalle neliöity vaakasuuntainen ja pystysuuntainen etäisyys ja lisätään niiden summan neliöjuuri muuttujaan euklidinen_arvo
        euklidinen_arvo += ((tavoitetila_x - tila_x)**2 + (tavoitetila_y - tila_y)**2)**0.5
    # Palautetaan lopuksi muuttuja euklidinen_arvo
    return euklidinen_arvo

Tarkistetaan, että euklidinen-etäisyys arviointifunktio palauttaa alkutilalle aiemmin lasketun arvon ~6,65.

In [8]:
# Määritetään kuvan 2 alkutila ja tavoitetila
alkutila = [7,2,3,1,0,6,5,8,4]
tavoitetila = [1,2,3,4,5,6,7,8,0]

# Tarkistetaan, että funktio laske_euklidinen_etäisyys() palauttaa oikean arvon
print('Euklidinen-etäisyys alkutilalle on: {}'.format(laske_euklidinen_etäisyys(alkutila, tavoitetila)))

Euklidinen-etäisyys alkutilalle on: 6.650281539872885


Tämän jälkeen ratkaistaan peli alkutilalle arviointifunktion ollessa euklidinen-etäisyys.

In [9]:
# Ratkaistaan peli alkutilalle arviointifunktion ollessa euklidinen-etäisyys.
ratkaise_peli(alkutila, tavoitetila, 'euklidinen')

Alkutila:
+---+---+---+
| 7 | 2 | 3 |
+---+---+---+
| 1 |   | 6 |
+---+---+---+
| 5 | 8 | 4 |
+---+---+---+


Ratkaisu alkutilalle A*-hakualgoritmilla euklidinen-etäisyys arviointifunktiota apuna käyttäen:

+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
| 4 | 5 | 6 |
+---+---+---+
| 7 | 8 |   |
+---+---+---+

Ratkaisu sisältää 18 liikettä, jonka löytämiseksi tutkittiin yhteensä 397 tilaa.


### Hamming-etäisyys arviointifunktio

Toteuta heuristinen hamming-etäisyys arviointifunktio.

In [ ]:
def laske_hamming_etäisyys(tila, tavoitetila):
    """
    Tämä funktio laskee hamming-etäisyyden tilalle tavoitetilasta.
    """
    #-------- TÄHÄN SINUN KOODI --------
    # Alusta muuttuja hamming_arvo arvolla 0
    
    # Käy silmukassa läpi laatan arvot 1-8
    
        # Selvitä tilalle ja tavoitetilalle, missä kohtaa LISTAA laatan arvo on (esim. tila_indeksi viittaa tilan laatan arvon indeksiin listassa. Vihje: .index())
        
        
        # Jos tilan laatan arvon indeksi on eri kuin tavoitetilan vastaavan laatan arvon indeksi
        
            # Kasvata muuttujan hamming_arvo arvoa 1:llä
            
    # Palauta lopuksi muuttuja hamming_arvo
    
    #-----------------------------------

Tarkistetaan, että hamming-etäisyys arviointifunktio palauttaa alkutilalle aiemmin lasketun arvon 4.

In [ ]:
# Tarkistetaan, että funktio laske_hamming_etäisyys() palauttaa oikean arvon
print('Hamming-etäisyys alkutilalle on: {}'.format(laske_hamming_etäisyys(alkutila, tavoitetila)))

Tämän jälkeen ratkaistaan peli alkutilalle arviointifunktion ollessa hamming-etäisyys.

In [ ]:
# Ratkaistaan peli alkutilalle arviointifunktion ollessa hamming-etäisyys.
ratkaise_peli(alkutila, tavoitetila, 'hamming')

### City-block-etäisyys arviointifunktio

Toteuta heuristinen city-block-etäisyys arviointifunktio.

In [ ]:
def laske_city_block_etäisyys(tila, tavoitetila):
    """
    Tämä funktio laskee city-block-etäisyyden tilalle tavoitetilasta.
    """
    #-------- TÄHÄN SINUN KOODISI --------
    # Alusta muuttuja city_block_arvo arvolla 0
    
    # Käy silmukassa läpi laatan arvot 1-8
    
        # Selvitä tilalle ja tavoitetilalle, missä kohtaa MATRIISIA laatan arvo on (esim. tila_x viittaa tilan laatan arvon vaakasuuntaiseen koordinaattiin)
        
        
        # Laske jokaiselle laatalle vaakasuuntainen ja pystysuuntainen itseisarvoistettu etäisyys ja lisää ne muuttujaan city-block_arvo
        
    # Palauta lopuksi muuttuja city_block_arvo
    
    #-------------------------------------

Tarkistetaan, että city-block-etäisyys arviointifunktio palauttaa alkutilalle aiemmin lasketun arvon 8.

In [ ]:
# Tarkistetaan, että funktio laske_city_block_etäisyys() palauttaa oikean arvon
print('City-block-etäisyys alkutilalle on: {}'.format(laske_city_block_etäisyys(alkutila, tavoitetila)))

Tämän jälkeen ratkaistaan peli alkutilalle arviointifunktion ollessa city-block-etäisyys.

In [ ]:
# Ratkaistaan peli alkutilalle arviointifunktion ollessa city-block-etäisyys.
ratkaise_peli(alkutila, tavoitetila, 'city-block')

**Mikä kolmesta käytetystä arviointifunktiosta on paras tälle ongelmalle tutkittujen tilojen määrän perusteella?**

`Tähän sinun vastaus`

# Palaute
Lopuksi vastaa seuraaviin kysymyksiin:

**Kuinka paljon aikaa käytit harjoituksen tekemiseen?**

`Tähän sinun vastaus`

**Kohtasitko ongelmia tai haasteita harjoitusta tehdessäsi? Oliko notebookissa riittävän kattava ohjeistus harjoituksen tekemiseen?**

`Tähän sinun vastaus`

**Jotain muuta palautetta harjoitukseen liittyen?**

`Tähän sinun vastaus`

# Palautus

1. Ennen kuin palautat tämän notebookin, **tarkista, että notebookin koodi toimii** ja tuottaa kaikki pyydetyt tulokset valitsemalla valikosta `Kernel -> Restart & Run All`! Tarkista lisäksi, että olet vastannut kaikkiin lihavoituihin **kysymyksiin.**
2. Tyhjennä kaikki tulosteet ja muuttujat valitsemalla valikosta `Kernel -> Restart & Clear Output`. Tämä välivaihe pienentää palautettavan notebookin kokoa merkittävästi! 
3. Nimeä uudelleen tämä notebook-tiedosto muodossa **`JT_H1_[opiskelijanumero(t)].ipynb`** (esim. `JT_H1_1234567.ipynb` jos teet harjoituksen yksin tai `JT_H1_1234567_2345678_3456789.ipynb` jos teette harjoituksen ryhmässä)
4. Palauta **ainoastaan** ratkaisemasi notebook-tiedosto (`.ipynb` päätteinen tiedosto) Moodleen Harjoitus 1 Hakualgoritmit. **Älä siis sisällytä työkansiota tai muita tiedostoja** palautukseesi! **HOX jokainen palauttaa tiedoston itsenäisesti moodleen**
5. Vastaa **Moodlessa harjoitus 1** henkilökohtaisesti vastaaviin kysymyksiin